In [1]:
from collections import defaultdict

import polars as pl
import math

In [2]:
df = pl.read_csv("clear.csv", columns=["Id_questions", "clean_question", "clean_answers", "Score_answers"])

In [3]:
df.head()

Id_questions,Score_answers,clean_question,clean_answers
i64,f64,str,str
80,12.0,"""written database generation sc…","""wound kind hack actually works…"
80,6.0,"""written database generation sc…","""sqlite api function called lik…"
80,1.0,"""written database generation sc…","""making delimiter little comple…"
90,13.0,"""good tutorials explaining bran…","""version control subversion goo…"
90,2.0,"""good tutorials explaining bran…","""try version control standalone…"


In [4]:
df_token = df.with_columns(pl.col("clean_question").str.split(" ").alias("tokens"))

In [5]:
df_token.head()

Id_questions,Score_answers,clean_question,clean_answers,tokens
i64,f64,str,str,list[str]
80,12.0,"""written database generation sc…","""wound kind hack actually works…","[""written"", ""database"", … ""statement""]"
80,6.0,"""written database generation sc…","""sqlite api function called lik…","[""written"", ""database"", … ""statement""]"
80,1.0,"""written database generation sc…","""making delimiter little comple…","[""written"", ""database"", … ""statement""]"
90,13.0,"""good tutorials explaining bran…","""version control subversion goo…","[""good"", ""tutorials"", … ""client""]"
90,2.0,"""good tutorials explaining bran…","""try version control standalone…","[""good"", ""tutorials"", … ""client""]"


In [6]:
df_token_NO_NAN = df_token.filter(
    ~pl.all_horizontal(pl.col('clean_question').is_null())
)

In [7]:
df_token_NO_NAN = df_token_NO_NAN.filter(
    ~pl.all_horizontal(pl.col('clean_answers').is_null())
)

In [8]:
df_token_NO_NAN.head()

Id_questions,Score_answers,clean_question,clean_answers,tokens
i64,f64,str,str,list[str]
80,12.0,"""written database generation sc…","""wound kind hack actually works…","[""written"", ""database"", … ""statement""]"
80,6.0,"""written database generation sc…","""sqlite api function called lik…","[""written"", ""database"", … ""statement""]"
80,1.0,"""written database generation sc…","""making delimiter little comple…","[""written"", ""database"", … ""statement""]"
90,13.0,"""good tutorials explaining bran…","""version control subversion goo…","[""good"", ""tutorials"", … ""client""]"
90,2.0,"""good tutorials explaining bran…","""try version control standalone…","[""good"", ""tutorials"", … ""client""]"


In [9]:
df_token_NO_NAN.null_count()

Id_questions,Score_answers,clean_question,clean_answers,tokens
u32,u32,u32,u32,u32
0,0,0,0,0


In [10]:
df_token_NO_NAN.head()

Id_questions,Score_answers,clean_question,clean_answers,tokens
i64,f64,str,str,list[str]
80,12.0,"""written database generation sc…","""wound kind hack actually works…","[""written"", ""database"", … ""statement""]"
80,6.0,"""written database generation sc…","""sqlite api function called lik…","[""written"", ""database"", … ""statement""]"
80,1.0,"""written database generation sc…","""making delimiter little comple…","[""written"", ""database"", … ""statement""]"
90,13.0,"""good tutorials explaining bran…","""version control subversion goo…","[""good"", ""tutorials"", … ""client""]"
90,2.0,"""good tutorials explaining bran…","""try version control standalone…","[""good"", ""tutorials"", … ""client""]"


In [11]:
count = df_token_NO_NAN.height

In [12]:
count

2013249

In [13]:
corpus = df_token_NO_NAN['clean_question'].to_list()
tokens = df_token_NO_NAN["tokens"].to_list()

In [14]:
invers_index = defaultdict(list)
df_counts = defaultdict(int)

for document_index, tokens_in_doc in enumerate(tokens):
    tf = defaultdict(int)
    for t in tokens_in_doc:
        tf[t] += 1
    for term, freq in tf.items():
        invers_index[term].append((document_index, freq))
        df_counts[term] += 1

In [15]:
idf = {t: math.log((count - df_counts[t] + 0.5) / (df_counts[t] + 0.5) + 1) for t in df_counts}

In [16]:
# Длина документов для BM25
doc_lens = [len(doc) for doc in tokens]
average_len = sum(doc_lens) / len(doc_lens)

In [17]:
def bm25_score(query, top_k=5, k1=1.5, b=0.75):

    if isinstance(query, str):
        query_tokens = query.split()
    else:
        query_tokens = query

    querry_tf = defaultdict(int)
    for t in query_tokens:
        querry_tf[t] += 1

    # Считаем BM25 для каждого документа
    scores = [0.0] * count

    for term, qf in querry_tf.items():
        if term not in invers_index:
            continue  # Если термина нет в инвертированном индексе, пропускаем

        # Получаем IDF для термина
        idf_term = idf.get(term, 0.0)

        # Для каждого документа, в котором встречается термин:
        for doc_idx, freq in invers_index[term]:
            doc_len = doc_lens[doc_idx]
            denom = freq + k1 * (1 - b + b * doc_len / average_len)
            numer = freq * (k1 + 1)
            score = idf_term * numer / denom
            scores[doc_idx] += score

    # Сортируем по убыванию и возвращаем top_k результатов
    ranked = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
    return ranked[:top_k]

In [18]:
# Функция для извлечения ответов на основе BM25
def retrieve_qa(query, top_k=5):
    results = bm25_score(query, top_k=top_k)
    qa = []
    for idx, _ in results:
        # Извлекаем question и соответствующий Id_questions
        question = df_token_NO_NAN[idx, "clean_question"]
        id_question = df_token_NO_NAN[idx, "Id_questions"]
        # Теперь находим все ответы, соответствующие этому вопросу
        answers = df_token_NO_NAN.filter(pl.col("Id_questions") == id_question)
        # Сортируем по рейтингу Score_a и выбираем лучший
        best_answer = answers.sort("Score_answers", descending=True).select("clean_answers").to_pandas().iloc[0][
            "clean_answers"]
        # Добавляем результат question, answer и Id_questions
        qa.append((id_question, question, best_answer))

    return qa

In [19]:
querry = "install sql"
querry_ans_res = retrieve_qa(querry, top_k=5)
for id_question, question, answer in querry_ans_res:
    print(f"Id_questions: {id_question}\nQ: {question}\nA: {answer}\n\n")

Id_questions: 5843820
Q: application language net database want create setup file dotnet framework wanna create setup like setup wizard running install frame work application pls tell sql database management install sql sever pcs option install sql server like net framework mean need install net language run application install framework like install run sql server thanks
A: thing application pre compiled net framework interpretes need run application decide want remain locally application needs centralized install application machines install instance data specific instance need package local application option centralized host application connects installation connect data needs


Id_questions: 2639310
Q: machine sql installed need install instance sql reasons main reason reporting services report servers sql create ssrs compatible need install sql server work need install second reason site team consulting want project upgraded rebuilding box safe way install box contains question s

In [20]:
querry = "How do you open a file in C++?"
querry_ans_res = retrieve_qa(querry, top_k=5)
for id_question, question, answer in querry_ans_res:
    print(f"Id_questions: {id_question}\nQ: {question}\nA: {answer}\n\n")

Id_questions: 4933710
Q: trying open file received argument store argument global variable open works successfully use open fails open file reason use strict use warnings file argv open fails open file file file argv works fine global file open file die open file specified error
A: unary open works package global variables documented manpage better way open file reading filename argv store argument variable open print file contents use strict warnings debugging perl scripts


Id_questions: 4933710
Q: trying open file received argument store argument global variable open works successfully use open fails open file reason use strict use warnings file argv open fails open file file file argv works fine global file open file die open file specified error
A: unary open works package global variables documented manpage better way open file reading filename argv store argument variable open print file contents use strict warnings debugging perl scripts


Id_questions: 4933710
Q: trying open f

In [21]:
querry = "Deploying SQL Server"
querry_ans_res = retrieve_qa(querry, top_k=5)
for id_question, question, answer in querry_ans_res:
    print(f"Id_questions: {id_question}\nQ: {question}\nA: {answer}\n\n")

Id_questions: 80
Q: written database generation script sql want execute adobe air application create table trole roleid integer primary key rolename varchar create table tfile fileid integer primary key filename varchar filedescription varchar thumbnailid integer fileformatid integer categoryid integer isfavorite boolean dateadded date globalaccesscount integer lastaccesstime date downloadcomplete boolean isnew boolean isspotlight boolean duration varchar create table tcategory categoryid integer primary key categoryname varchar parent categoryid integer execute adobe air following methods public static function runsqlfromfile filename string void var file file file applicationdirectory resolvepath filename var stream filestream new filestream stream open file filemode read var strsql string stream readutfbytes stream bytesavailable nonquery strsql public static function nonquery strsql string void var sqlconnection sqlconnection new sqlconnection sqlconnection open file applicationsto

In [22]:
def precision_at_k(search_fn, queries, ground_truth, k=5):
    precisions = []
    for q, true_id in zip(queries, ground_truth):
        # Получаем топ k документов для запроса
        results = search_fn(q, top_k=k)
        ids = [df_token_NO_NAN[idx, "Id_questions"] for idx, _ in results]
        # Проверяем, попал ли true_id в топ k
        precisions.append(int(true_id in ids) / k)
    return sum(precisions) / len(precisions)

In [23]:
test_qs = ["install sql", "Deploying SQL Server", "How do you open a file in C++?"]
test_gt = [2639310, 90, 4933710]

Оценка Precision@5 показала, что поиск не нашёл правильный айдишники ответов =(

In [24]:
print("Precision@5:", precision_at_k(bm25_score, test_qs, test_gt, k=5))

Precision@5: 0.20000000000000004


In [25]:
def mrr(search_fn, queries, ground_truth):
    rr = []
    for q, true_id in zip(queries, ground_truth):
        results = search_fn(q, top_k=len(df_token_NO_NAN))
        ids = [df_token_NO_NAN[idx, "Id_questions"] for idx, _ in results]

        if true_id in ids:
            rank = ids.index(true_id) + 1
            rr.append(1.0 / rank)
        else:
            rr.append(0.0)

    return sum(rr) / len(rr)

Функция рассчитывает Mean Reciprocal Rank (MRR), где search_fn - функция поиска, queries - список запросов, ground_truth - список правильных ID вопросов.

In [26]:
print("MRR:", mrr(bm25_score, test_qs, test_gt))

MRR: 0.5833333333333334


MMR показало, что поиск работает плохо


# Результаты

* Система анализирует запрос, токенизирует его, затем вычисляет BM25-оценки для каждого вопроса. Результаты сортируются по убыванию и возвращаются пользователю.

    * Используя формулу BM25, для каждого запроса вычисляем его сходство с каждым документом.

    $$ BM25(t, d) = IDF(t) \cdot \frac{TF(t, d) \cdot (k_1 + 1)}{TF(t, d) + k_1 \cdot (1 - b + b \cdot \frac{|d|}{avgdl})}$$

    * После нахождения наиболее релевантных вопросов, извлечены лучшие ответы с использованием рейтинга со стековерфлоу для каждого вопроса.

* Создал инвертированный индекс, где каждому термину сопоставляется список документов, содержащих этот термин, с его частотой в каждом документе.

* Для каждого запроса вычисляем частоты: TF (Term Frequency) для термина в документе и IDF (Inverse Document Frequency), что позволяет взвесить частоту термина в документе с учётом его распространённости во всем документе.

* Precision@5: Это метрика, измеряющая точность среди топ-5 результатов. В нашем случае система не получила правильного ответа, значение - мало.

* MRR (Mean Reciprocal Rank): Это средний показатель обратного ранга правильного ответа среди всех запросов.


### Плюсы:

* Быстрая обработка запросов: использование инвертированного индекса для поиска термина в документах значительно ускоряет выполнение.

* Простота: BM25 является относительно простым и эффективным методом для оценки релевантности документов.

### Минусы:

* Отсутствие учёта контекста и синонимов: BM25 основывается только на частотах терминов в документах, и не учитывает, например, смысловых связей между словами или синонимами.
